# Python interface to connect to V-REP

In [2]:
# Libraries from Jupyter Notebook Python

%matplotlib notebook

import numpy as np
import matplotlib.pylab as plt

import vrep #imports vrep file
import time #time functions

import cvxopt #for resolved rate control algorithm

#functions that allow to easily start a connection with the VREP software and get positions of the robot
from vrep_utils import start_vrep_connection, initialize_communication, getPoseWorldFrame

np.set_printoptions(precision=5,linewidth=120,suppress=True)

ModuleNotFoundError: No module named 'cvxopt'

# Connection to V-REP and usage examples to get the state of the robot

In [ ]:
# Create the connection with V-REP
client_id = start_vrep_connection()

# Get the joints and effector handles 
joint_handles, end_effector_handle = initialize_communication(client_id)

# The endeffector position in the world frame of Vrep
end_effector_pose = getPoseWorldFrame(client_id, end_effector_handle)
print '\n the pose of the end effector is'
print end_effector_pose

# The joint pose in the world frame of Vrep (the angles)
joint_positions = np.zeros([7,1])
for i in range(7):
    res, joint = vrep.simxGetJointPosition(client_id,joint_handles[i],vrep.simx_opmode_buffer)
    joint_positions[i] = joint
print '\n the position of the joints are'
print joint_positions


In [ ]:
def Jacobian(theta, end_eff_pose):
    j = np.zeros([3,len(theta)]) #Initialize the Jacobian matrix that is dependant on the number of pose (from inverse kinematics.jpynb)
    for i in range(len(theta)):
        w = theta[i][0:3,2] #Calculate the angular velocity (omega)
        d = theta[i][0:3,3] - end_eff_pose[0:3,3] #calculate the distance between the endeffector and the joint (of the Jacobian)
        j[:,i] = np.cross(d,w) #The top 3 by N jacobian matrix, since all of the joints are rotary joint (no prismatic joint)
                               #Hence the top 3 by N jacobian matrix = (Pob - Poe) x w 
    return j

In [ ]:
#5th order Polynomial Trajectory Generation 
def get_coef_(xs, vxs, axs, xe, vxe,axe, T):
    #This A matrix is created because of the initial wanted condition where s(0)=v(0)=a(0)=v(t)=a(t)=0 and s(t) = xe (desired position)
    #Hence the A matrix become: (explanation on the document)
    
    A = np.array([[1, 0, 0, 0,0,0],
                [1, T, T**2, T**3,T**4,T**5],
                [0, 1, 0, 0, 0, 0],
                [0, 1, 2*T, 3*T**2, 4*T**3, 5*T**4],
                [0,0,2,0,0,0],
                [0,0,2,6*T,12*T**2,20*T**3]])
        
    b = np.array([xs, xe, vxs, vxe, axs, axe])
    x = np.linalg.solve(A, b)
# since x = [a0,a1,a2,a3,a4,a5], hence x can be calculated as x=A(inverse) b
    a0 =x[0]
    a1 = x[1]
    a2 = x[2]
    a3 = x[3]
    a4 = x[4]
    a5 = x[5]
    
    return a0,a1,a2,a3,a4,a5

#calculating position, velocity and acceleration
def calc_point(a0,a1,a2,a3,a4,a5,t):
    xt = a0 + a1 * t + a2 * t**2 + a3 * t**3 + a4 * t**4 + a5 * t**5

    return xt

def calc_first_derivative(a0,a1,a2,a3,a4,a5,t):
    xt = a1 + 2 * a2 * t + 3 * a3 * t**2 + 4 * a4 * t**3 + 5 * a5 * t**4

    return xt
   
def calc_second_derivative(a0,a1,a2,a3,a4,a5,t):
    xt = 2 * a2 + 6 * a3 * t + 12 * a4 * t**2 + 20 * a5 * t**3

    return xt
    
def poly5_trajectory (sx, sy, sz, sxv, syv, szv, sxa, sya, sza, gx, gy,gz, gxv, gyv, gzv,gxa,gya, gza, dt, Min_T, Max_T):
    
    #Consider all the possible deadline that the robot needs to reach: (detailed on the document)
    for T in np.arange(Min_T, Max_T):
            xqp = get_coef_(sx , sxv, sxa, gx, gxv, gxa, T)
            yqp = get_coef_(sy , syv, sya, gy, gyv, gya, T)
            zqp = get_coef_(sz , szv, sza, gz, gzv, gza, T)

            time, rx, ry, rz, rv, ra = [], [], [], [], [], []
            #recalculting the possible values of distance, speed and velocity
            for t in np.arange(0.0, T + dt, dt):
                time.append(t)
                rx.append(calc_point(xqp[0],xqp[1],xqp[2],xqp[3],xqp[4],xqp[5],t))
                ry.append(calc_point(yqp[0],yqp[1],yqp[2],yqp[3],yqp[4],yqp[5],t))
                rz.append(calc_point(zqp[0],zqp[1],zqp[2],zqp[3],zqp[4],zqp[5],t))

                vx = calc_first_derivative(xqp[0],xqp[1],xqp[2],xqp[3],xqp[4],xqp[5],t)
                vy = calc_first_derivative(yqp[0],yqp[1],yqp[2],yqp[3],yqp[4],yqp[5],t)
                vz = calc_first_derivative(zqp[0],zqp[1],zqp[2],zqp[3],zqp[4],zqp[5],t)
                rv = np.stack((vx, vy,vz), axis=-1)

                ax = calc_second_derivative(xqp[0],xqp[1],xqp[2],xqp[3],xqp[4],xqp[5],t)
                ay = calc_second_derivative(yqp[0],yqp[1],yqp[2],yqp[3],yqp[4],yqp[5],t)
                az = calc_second_derivative(zqp[0],zqp[1],zqp[2],zqp[3],zqp[4],zqp[5],t)
                ra = np.stack((ax, ay,az), axis=-1)
                
                
    return rx[-5:-1], ry[-5:-1], rz[-5:-1], rv[-5:-1], ra[-5:-1], ra[-5:-1] 
#there are many possibile path to go, I will only create a B spline of the last 5 values. 
#In practice this 5 points should be distance propotionally to create a good control, an example (1/5*goal, 2/5*goal, 3/5*start, 4/5*goal, goal)

In [ ]:
# each control cycle is dt = 0.05 second 
# each control cycle works as follows: 1) read sensors, 2) compute control, 3) send desired position and velocity to robot
t = 0.0
dt = 0.05
T = 5.0

x_des = [0.65,-0.15,0.57] #The desired position (position of the cup)

#Trajectory generation for given object position with condition s(0)=v(0)=a(0)=v(t)=a(t)=0 and s(t) = xe (desired position)
test = poly5_trajectory (0, 0, 0, 0,0,0, 0,0,0, x_des[0], x_des[1],x_des[2], 0, 0, 0,0,0,0, 0.05, T-3, T)

# save the recorded trajectories and display for analy
saved_pos_trajectory = np.zeros([int(T/dt),7])
saved_current_pos = np.zeros([int(T/dt),7])
saved_vel_trajectory = np.zeros([int(T/dt),7])
default_position = np.array([0., 0., 0., 0., 0., 0., 0.])
# set the desired joint positions of the robot as the actual ones
desired_position = np.zeros(7)
desired_velocity = np.zeros(7)
for i in range(7):
    res, joint = vrep.simxGetJointPosition(client_id,joint_handles[i],vrep.simx_opmode_buffer)
    desired_position[i] = joint

# Run the control loop for T/dt step (i.e. for T seconds)
for steps in range(int(T/dt)):
    # some useful variables we will need
    current_joint_poses = []
    current_joint_angles = np.zeros([7])
    
    # get the position of all the joints angles and their pose in world frame
    for i in range(7):
        res, joint = vrep.simxGetJointPosition(client_id,joint_handles[i],vrep.simx_opmode_buffer)
        current_joint_angles[i] = joint
        current_joint_poses.append(getPoseWorldFrame(client_id, joint_handles[i]))
   
    #calculatethe inverse kinematics for every desired spline trajectory
    for i in range (4):
        trajectory=[test[0][i], test[1][i],test[2][i]]
        #get the end-effector pose
        current_end_effector_pose = getPoseWorldFrame(client_id, end_effector_handle)
        #current end-effector position (this will be used in the IK control)
        current_end_effector_position = current_end_effector_pose[0:3,3]
        jacobian = Jacobian(current_joint_poses, current_end_effector_pose)

        # Inverse Kinematics Rate of Control Algorithm in Inverse Kinematics.jpynb This inverse kinematics is controlling the norm of theta.dot
        

        cvxopt.solvers.options['show_progress'] = False #do not show verbose information for the QP solver

        x_current = current_end_effector_position #get the current position 

        error = 1.1 *((trajectory - x_current)) #compute the error times a small gain

        Q = cvxopt.matrix(np.identity(10)) #1/2 xT Q x -> the resulting Q matrix will be 10 by 10 with elements as Identity.
        Q[0:7,0:7] *= dt*dt
        Q[7:10, 7:10] *= 10
        Q[1,1] *= 100.
        q = cvxopt.matrix(np.concatenate((dt*(desired_position-default_position),np.zeros(3)),axis=0)) #subject to become zero
        q[1] *= 10

        A = cvxopt.matrix(np.concatenate((jacobian, np.identity(3)), axis=1)) #A is subject to the identity
        b = cvxopt.matrix(error) #subject to P(xdesired-x) = error
        G = cvxopt.matrix(np.zeros([14,10])) #subject to become zero
        h = cvxopt.matrix(np.ones([14,1])*0.3) #subject to Gx<= h

        sol = cvxopt.solvers.qp(Q,q,G,h,A,b,verbose=False)
        #calculating desired velocity and position
        velocity_desired = (np.array(sol['x']).transpose()[0]).transpose()[0:7] #get back the solution into a np.array we like
        desired_position += dt * velocity_desired

        #this is done to improve communication efficiency by sending everything in batch
        vrep.simxPauseCommunication(client_id,True)

        #send the desired position and velocities to the joints
        for i in range(7):
            vrep.simxSetJointTargetPosition(client_id,joint_handles[i],desired_position[i],vrep.simx_opmode_streaming)
            vrep.simxSetJointTargetVelocity(client_id,joint_handles[i],desired_velocity[i],vrep.simx_opmode_streaming)

        # resume communication
        vrep.simxPauseCommunication(client_id,False) #resume communication

        # keep track to time
        t += dt

        #save desired and current trajectories
        saved_pos_trajectory[steps,:] = desired_position
        saved_vel_trajectory[steps,:] = desired_velocity
        saved_current_pos[steps,:] = current_joint_angles

        #sleep to wait for the next control cycle
        time.sleep(dt)

#plot the results
#plot desired vs. actual joint angles
plt.figure()
for i in range(7):
    plt.subplot(7,1,i+1)
    plt.plot(saved_pos_trajectory[:,i])
    plt.plot(saved_current_pos[:,i])